# Redo experiments on the graphs from the paper

In [ ]:
import sys
sys.path.append("../../gershgorin/")

In [ ]:
import networkx as nx
from pygsp import graphs
import numpy as np
import matplotlib.pyplot as plt
import scipy
import time

from src.gershgorin.bs_gda import BS_GDA
from src.graph.graph import Graph

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

### Experimental setup

#### Graphs

We create four types of graphs which are all available in the PyGSP toolbox. Parameters are chosen as in the paper.

In [ ]:
seed = 2022
n = 100
sensor_graph = graphs.Sensor(n, seed=seed)
community_graph = graphs.Community(n, seed=seed)
barabasi_albert_graph = graphs.BarabasiAlbert(n, m=1, seed=seed)
barabasi_albert_graph.set_coordinates(kind='spring', seed=seed)
minnesota_graph = graphs.Minnesota()

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(10,10))
sensor_graph.plot(ax=axes[0][0], vertex_size=50)
community_graph.plot(ax=axes[0][1], vertex_size=50)
barabasi_albert_graph.plot(ax=axes[1][0], vertex_size=50)
minnesota_graph.plot(ax=axes[1][1], vertex_size=40)

#### Weight matrices

Edge weights are computed using a **gaussian similarity** function:
$$
w_{ij} = \exp \left( -\frac{||x_i - x_j||^2_2}{\sigma^2_x} \right)
$$

- random sensor graphs: weighted graphs (nothing to do)
- rest are unweighted graphs $\rightarrow$ compute weights manually
- community: set $\sigma_x = 1$
- Minnesota: set $\sigma_x=0.1$
- Barabasi-Albert: generate random uniform edge weights from interval $(0,1)$

In [ ]:
from scipy.spatial.distance import pdist, squareform

def gaussian_similarity(X, sigma):
    D = squareform(pdist(X, 'euclidean'))
    return np.exp(-(D**2) / sigma**2)

In [ ]:
W_sensor = sensor_graph.W
W_community = gaussian_similarity(community_graph.coords, 1.) * (np.asarray(community_graph.W.todense()) > 0)
W_minnesota = gaussian_similarity(minnesota_graph.coords, 0.1) * (np.asarray(minnesota_graph.W.todense()) > 0)
W_barabasi_albert = np.random.uniform(0, 1, barabasi_albert_graph.W.shape)

### Cycle graph

In [ ]:
cycle_graph = nx.cycle_graph(20)
G = Graph(nx.adjacency_matrix(cycle_graph).todense())
sampling_budget = 5
mu = 0.01
epsilon = 1e-5
tic = time.process_time()
sampling_set, T = BS_GDA().bs_gda(G, sampling_budget, mu, epsilon)
print(f"This took {time.process_time()-tic} s")
print(f"Sampled {len(sampling_set)} nodes")

In [ ]:
c = np.zeros(G.num_nodes)
c[list(sampling_set)] = 1
plt.figure(figsize=(4,4))
nx.draw(cycle_graph, pos=nx.layout.circular_layout(cycle_graph), node_size=30, width=0.6, node_color=c)

### Connected caveman graph

In [ ]:
cavemen_graph = nx.connected_caveman_graph(10, 5)
G = Graph(nx.adjacency_matrix(cavemen_graph).todense())
sampling_budget = 10
mu = 0.01
epsilon = 1e-5
tic = time.process_time()
sampling_set, T = BS_GDA().bs_gda(G, sampling_budget, mu, epsilon)
print(f"This took {time.process_time()-tic} s")
print(f"Sampled {len(sampling_set)} nodes")

In [ ]:
c = np.zeros(G.num_nodes)
c[list(sampling_set)] = 1
plt.figure(figsize=(4,4))
nx.draw(cavemen_graph, pos=nx.kamada_kawai_layout(cavemen_graph), node_size=30, width=0.6, node_color=c)

### Community graph

In [ ]:
G = nx.Graph(community_graph.W)
# take largest connected component
S = G.subgraph(max(nx.connected_components(G), key=len))
G = Graph(nx.adjacency_matrix(S))
sampling_budget = 5
mu = 0.01
epsilon = 1e-5
tic = time.process_time()
sampling_set, T = BS_GDA().bs_gda(G, sampling_budget, mu, epsilon)
print(f"This took {time.process_time()-tic} s")
print(f"Sampled {len(sampling_set)} nodes")

In [ ]:
c = np.zeros(G.num_nodes)
c[list(sampling_set)] = 1
plt.figure(figsize=(4,4))
nx.draw(S, pos=community_graph.coords, node_size=c*50, width=0.3, node_color=c)

### Sensor graph

In [ ]:
G = Graph(sensor_graph.W)
sampling_budget = int(0.1*G.num_nodes)
mu = 0.01
epsilon = 1e-5
tic = time.process_time()
sampling_set, T = BS_GDA().bs_gda(G, sampling_budget, mu, epsilon)
print(f"This took {time.process_time()-tic} s")
print(f"Sampled {len(sampling_set)} nodes")

In [ ]:
c = np.zeros(G.num_nodes)
c[list(sampling_set)] = 1
plt.figure(figsize=(4,4))
nx.draw(nx.Graph(sensor_graph.W), pos=sensor_graph.coords, node_size=30, width=0.6, node_color=c)

### Barabasi-Albert graph

In [ ]:
G = Graph(barabasi_albert_graph.W)
sampling_budget = int(0.1*G.num_nodes)
mu = 0.01
epsilon = 1e-5
tic = time.process_time()
sampling_set, T = BS_GDA().bs_gda(G, sampling_budget, mu, epsilon)
print(f"This took {time.process_time()-tic} s")
print(f"Sampled {len(sampling_set)} nodes")

In [ ]:
c = np.zeros(G.num_nodes)
c[list(sampling_set)] = 1
plt.figure(figsize=(4,4))
nx.draw(nx.Graph(barabasi_albert_graph.W), pos=barabasi_albert_graph.coords, node_size=30, width=0.6, node_color=c)

### Minnesota graph

In [ ]:
G = Graph(minnesota_graph.W)
sampling_budget = int(0.1*G.num_nodes)
mu = 0.01
epsilon = 1e-5
tic = time.process_time()
sampling_set, T = BS_GDA().bs_gda(G, sampling_budget, mu, epsilon)
print(f"This took {time.process_time()-tic} s")
print(f"Sampled {len(sampling_set)} nodes")

In [ ]:
c = np.zeros(G.num_nodes)
c[list(sampling_set)] = 1
plt.figure(figsize=(4,4))
nx.draw(nx.Graph(minnesota_graph.W), pos=minnesota_graph.coords, node_size=20, width=0.6, node_color=c)